In [80]:
import math
import os
import random
import shutil
import pickle
import matplotlib.colors
import matplotlib.gridspec as gridspec
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import warnings
import tensorflow as tf
import shap
import sys

from scipy.stats import ttest_ind
from scipy.special import softmax
from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer

SEED_VALUE = 11
os.environ['PYTHONHASHSEED']=str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
tf.random.set_seed(SEED_VALUE)
warnings.filterwarnings('ignore')
os.chdir(os.getcwd())

In [94]:
with open("assets/results/cnn_model-WORD2VEC-STATIC-EARLY_STOPPING-TEST.pickle", 'rb') as handle:
    results_base = pickle.load(handle)
    results_base = pd.DataFrame(results_base)
    results_one_rule = results_base.loc[results_base["rule_label"]==1]
sentiment_label = np.array(list(results_one_rule["sentiment_label"]))
sentiment_pred = np.array(list(results_one_rule["sentiment_prediction_output"]))
corrects = np.where((sentiment_label==sentiment_pred)==True, 1, 0).tolist()
print(sum(corrects)/len(corrects))

0.7666666666666667


In [22]:
def getStats(dist):
    m = np.mean(dist)
    std = np.std(dist)
    h = 1.96*std/math.sqrt(len(dist))
    return m, h

labels = ['word2vec', 'Glove', 'ELMo', 'BERT']
width = 0.6

In [19]:
corrects_distribution = pd.read_csv("assets/existing_results_in_paper/results/corrects_distribution/rule_test_set.csv")

w2v_static_corrects = corrects_distribution['CNN_word2vec_static_base_no_project']
dist_w2v_static_corrects = corrects_distribution['Distilled_CNN_word2vec_static_base_no_project']

w2v_non_static_corrects = corrects_distribution['CNN_word2vec_non-static_base_no_project']
dist_w2v_non_static_corrects = corrects_distribution['Distilled_CNN_word2vec_non-static_base_no_project']

glove_static_corrects = corrects_distribution['CNN_glove_static_base_no_project']
dist_glove_static_corrects = corrects_distribution['Distilled_CNN_glove_static_base_no_project']

glove_non_static_corrects = corrects_distribution['CNN_glove_non-static_base_no_project']
dist_glove_non_static_corrects = corrects_distribution['Distilled_CNN_glove_non-static_base_no_project']

bert_static_corrects = corrects_distribution['CNN_bert_static_base_no_project']
dist_bert_static_corrects  = corrects_distribution['Distilled_CNN_bert_static_base_no_project']

bert_non_static_corrects = corrects_distribution['CNN_bert_non-static_base_no_project']
dist_bert_non_static_corrects  = corrects_distribution['Distilled_CNN_bert_non-static_base_no_project']

elmo_static_corrects = corrects_distribution['CNN_elmo_static_base_no_project']
elmo_non_static_corrects = corrects_distribution['CNN_elmo_non-static_base_no_project']


dist_elmo_non_static_corrects = corrects_distribution['Distilled_CNN_elmo_non-static_base_no_project']
dist_elmo_static_corrects = corrects_distribution['Distilled_CNN_elmo_static_base_no_project']

distribution_accuracy = {"w2v_static":w2v_static_corrects,
                "dist_w2v_static":dist_w2v_static_corrects,
                "w2v_non_static":w2v_non_static_corrects,
                "dist_w2v_non_static":dist_w2v_non_static_corrects,
                "glove_static":glove_static_corrects,
                "dist_glove_static":dist_glove_static_corrects,
                "glove_non_static":glove_non_static_corrects,
                "dist_glove_non_static":dist_glove_non_static_corrects,
                "elmo_static":elmo_static_corrects,
                "elmo_non_static":elmo_non_static_corrects,
                "bert_static":bert_static_corrects,
                "dist_bert_static":dist_bert_static_corrects,
                "bert_non_static":bert_non_static_corrects,
                "dist_bert_non_static":dist_bert_non_static_corrects}

distribution_accuracy_dataframe = pd.DataFrame(distribution_accuracy)


In [20]:
distribution_accuracy_dataframe

,w2v_static,dist_w2v_static,w2v_non_static,dist_w2v_non_static,glove_static,dist_glove_static,glove_non_static,dist_glove_non_static,elmo_static,elmo_non_static,bert_static,dist_bert_static,bert_non_static,dist_bert_non_static
0,1,1,1,1,0,1,1,1,0,1,1,1,1,1
1,0,0,0,1,0,0,0,0,0,1,1,1,0,0
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,0,0,0,0,0,0,0,0,0,1
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4307,0,1,1,1,1,1,1,0,1,1,1,1,1,1
4308,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4309,0,0,0,0,0,0,0,0,0,1,1,1,1,1
4310,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [28]:
accuracies = [w2v_static_corrects, w2v_non_static_corrects] 
correct_means = [getStats(x)[0] for x in accuracies]
correct_std = [getStats(x)[1] for x in accuracies]
correct_means

[0.7460575139146568, 0.7704081632653061]